# ODBC

Stands for __O__pen __D__atabase __C__onnectivity. It provides an __A__pplication __P__rogramming __I__nterface (API) that allows client side programs to call DBMS.<br>
* Query and transaction requests are sent using the ODBC API to the RDBMS.
* Query results are sent back to client programs.
* Example: JDBC for Java programs, pyodbc for Python.<br>
<img src="arch.jpg" alt="Three tier client server architecture for DBMS" title="Three tier architecture" width="400" hight="400" align="left"/>

Some guidelines for writing Jupyter notebooks can be found here <br>
https://www.ibm.com/support/knowledgecenter/SSHGWL_1.2.3/analyze-data/markd-jupyter.html

# Jupyter notebook

To work with jupyter notebook: <br>
* Install Anaconda https://www.anaconda.com/
* Install pyodbc as illustrated here https://pypi.org/project/pyodbc/ by typing pip install pyodbc in anaconda prompt (you will find anaconda prompt in the start menu)
* Launch jupyter notebook from anaconda prompt by typing: jupyter notebook

# Imports

In [160]:
import pyodbc as podbc
import pandas as pd
# More about Pandas https://pandas.pydata.org/about/index.html

# Connect to DB method

In [161]:
def connect_to_db():
    conn = podbc.connect('Driver={SQL Server};'
                          'Server=RAISA-CAIRO-19\SQLEXPRESS;'
                          'Database={COMPANY};'
                          'Trusted_Connection=yes;')
    cursor = conn.cursor()
    return conn, cursor

# Execute query method

In [162]:
def execute_query(query, cursor):
    cursor.execute(query)
    for row in cursor:
        print(row)
    df = pd.read_sql_query(query, conn)
    cursor.commit()
    return df

In [244]:
def execute_non_return_query(query, cursor):
    cursor.execute(query)
    cursor.commit()

# Connect to DB

In [164]:
conn, cursor = connect_to_db()

# Create DB

In [249]:
query = """CREATE DATABASE COMPANY"""

In [ ]:
execute_non_return_query(query, cursor)

# Create table query

In [207]:
query = """DROP TABLE IF EXISTS EMPLOYEE"""

In [208]:
execute_non_return_query(query, cursor)

In [209]:
query = """DROP TABLE IF EXISTS DEPARTMENT"""

In [210]:
execute_non_return_query(query, cursor)

In [211]:
query = """CREATE TABLE DEPARTMENT(
Dname VARCHAR(15) NOT NULL,
Dnumber INT NOT NULL CHECK (Dnumber>0 AND Dnumber<21),
PRIMARY KEY(Dnumber)
)
"""

In [212]:
print(query)

CREATE TABLE DEPARTMENT(
Dname VARCHAR(15) NOT NULL,
Dnumber INT NOT NULL CHECK (Dnumber>0 AND Dnumber<21),
PRIMARY KEY(Dnumber)
)



In [213]:
execute_non_return_query(query, cursor)

In [214]:
# IMPORTANT: in order to specify on delete set default, default must be specified as in the table below
# On delete cascade will delete all rows with a foreign key referencing a primary key in another table, on update cascade will 
# update value of the foreign key if the primary key is updated.
query = """CREATE TABLE EMPLOYEE(
Fname VARCHAR(15) NOT NULL,
Minit CHAR,
Lname VARCHAR(15) NOT NULL,
Ssn CHAR(9) NOT NULL,
Bdate DATE,
Address VARCHAR(30),
gender CHAR,
Salary DECIMAL(10,2),
Super_ssn CHAR(9),
Dno int NOT NULL DEFAULT 1,
PRIMARY KEY(Ssn),
CONSTRAINT EMPDEPTFK FOREIGN KEY(Dno) REFERENCES DEPARTMENT(Dnumber) ON DELETE SET DEFAULT ON UPDATE CASCADE
)"""

In [215]:
execute_non_return_query(query, cursor)

In [216]:
query = """ALTER TABLE EMPLOYEE ADD CONSTRAINT SUPERSSNFK FOREIGN KEY(Super_ssn) REFERENCES EMPLOYEE(Ssn)"""

In [217]:
execute_non_return_query(query, cursor)

# Insert

In [236]:
query = """INSERT INTO DEPARTMENT VALUES('CS', 2)"""

In [237]:
execute_non_return_query(query, cursor)

In [220]:
query = """INSERT INTO EMPLOYEE VALUES('John', 'B', 'Smith', '123456789', '1965-01-09', 
'371 Fondren, Housten, TX', 'M', 30000, NULL, 1)
"""

In [221]:
execute_non_return_query(query, cursor)

In [222]:
query = """INSERT INTO EMPLOYEE VALUES('Ahmed', 'S', 'Taher', '987654321', '1965-01-09', 
'371 Fondren, Housten, TX', 'M', 30000, '123456789', 1)
"""

In [223]:
execute_non_return_query(query, cursor)

# Select

In [224]:
query = "SELECT * FROM EMPLOYEE"

In [225]:
execute_query(query, cursor)

('John', 'B', 'Smith', '123456789', '1965-01-09', '371 Fondren, Housten, TX', 'M', Decimal('30000.00'), None, 1)
('Ahmed', 'S', 'Taher', '987654321', '1965-01-09', '371 Fondren, Housten, TX', 'M', Decimal('30000.00'), '123456789', 1)


,Fname,Minit,Lname,Ssn,Bdate,Address,gender,Salary,Super_ssn,Dno
0,John,B,Smith,123456789,1965-01-09,"371 Fondren, Housten, TX",M,30000.0,None,1
1,Ahmed,S,Taher,987654321,1965-01-09,"371 Fondren, Housten, TX",M,30000.0,123456789,1


In [226]:
query = """DROP TABLE IF EXISTS D1EMP"""

In [227]:
execute_non_return_query(query, cursor)

In [228]:
query = """
SELECT * INTO D1EMP
FROM EMPLOYEE
WHERE Dno = 1
"""

In [229]:
execute_non_return_query(query, cursor)

In [230]:
query = "SELECT * FROM D1EMP"

In [231]:
execute_query(query, cursor)

('John', 'B', 'Smith', '123456789', '1965-01-09', '371 Fondren, Housten, TX', 'M', Decimal('30000.00'), None, 1)
('Ahmed', 'S', 'Taher', '987654321', '1965-01-09', '371 Fondren, Housten, TX', 'M', Decimal('30000.00'), '123456789', 1)


,Fname,Minit,Lname,Ssn,Bdate,Address,gender,Salary,Super_ssn,Dno
0,John,B,Smith,123456789,1965-01-09,"371 Fondren, Housten, TX",M,30000.0,None,1
1,Ahmed,S,Taher,987654321,1965-01-09,"371 Fondren, Housten, TX",M,30000.0,123456789,1


In [232]:
query = "SELECT ALL SALARY FROM EMPLOYEE"

In [233]:
execute_query(query, cursor)

(Decimal('30000.00'), )
(Decimal('30000.00'), )


,SALARY
0,30000.0
1,30000.0


In [242]:
query = "SELECT * FROM EMPLOYEE, DEPARTMENT"

In [243]:
execute_query(query, cursor)

('John', 'B', 'Smith', '123456789', '1965-01-09', '371 Fondren, Housten, TX', 'M', Decimal('30000.00'), None, 1, 'IS', 1)
('Ahmed', 'S', 'Taher', '987654321', '1965-01-09', '371 Fondren, Housten, TX', 'M', Decimal('30000.00'), '123456789', 1, 'IS', 1)
('John', 'B', 'Smith', '123456789', '1965-01-09', '371 Fondren, Housten, TX', 'M', Decimal('30000.00'), None, 1, 'CS', 2)
('Ahmed', 'S', 'Taher', '987654321', '1965-01-09', '371 Fondren, Housten, TX', 'M', Decimal('30000.00'), '123456789', 1, 'CS', 2)


,Fname,Minit,Lname,Ssn,Bdate,Address,gender,Salary,Super_ssn,Dno,Dname,Dnumber
0,John,B,Smith,123456789,1965-01-09,"371 Fondren, Housten, TX",M,30000.0,None,1,IS,1
1,Ahmed,S,Taher,987654321,1965-01-09,"371 Fondren, Housten, TX",M,30000.0,123456789,1,IS,1
2,John,B,Smith,123456789,1965-01-09,"371 Fondren, Housten, TX",M,30000.0,None,1,CS,2
3,Ahmed,S,Taher,987654321,1965-01-09,"371 Fondren, Housten, TX",M,30000.0,123456789,1,CS,2
